In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import dmc
from process import processed_data

Tensorflow not installed


In [3]:
data = processed_data()
train_ids, test_ids = dmc.loading.load_ids('rawMirrored')
train, test = dmc.preprocessing.split_train_test(data, train_ids, test_ids)

## SVM evaluation using classic ensemble splits

In [4]:
# allocate memory
data = train_ids = test_ids = None
####
ensemble = dmc.ensemble.Ensemble(train, test)

In [5]:
for split in ensemble.splits:
    print(len(ensemble.splits[split][1]), split)

2082 unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup
14 unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup
22160 unknown_articleID-unknown_customerID-known_voucherID-known_productGroup
3239 unknown_articleID-known_customerID-unknown_voucherID-known_productGroup
12 unknown_articleID-known_customerID-known_voucherID-unknown_productGroup
62265 unknown_articleID-known_customerID-known_voucherID-known_productGroup
4189 known_articleID-unknown_customerID-unknown_voucherID-known_productGroup
42844 known_articleID-unknown_customerID-known_voucherID-known_productGroup
6496 known_articleID-known_customerID-unknown_voucherID-known_productGroup
93197 known_articleID-known_customerID-known_voucherID-known_productGroup


Print splits and names to know where to not drop customer for naive bayes.

In [6]:
def shuffle(df: pd.DataFrame) -> pd.DataFrame:
    return df.reindex(np.random.permutation(df.index))

In [7]:
for split in ensemble.splits:
    ensemble.splits[split] = (shuffle(ensemble.splits[split][0])[:50000], ensemble.splits[split][1])

In [8]:
splits = len(ensemble.splits)
scalers = [dmc.transformation.scale_raw_features] * splits
ignore = [None] * splits
ensemble.transform(scalers=scalers, ignore_features=ignore)

/home/alex/anaconda3/lib/python3.5/site-packages/scipy/sparse/compressed.py:739: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [9]:
res = []
for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.SVM
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    res.append((k, prec, len(ye)))
    print(k, prec, len(ye))

unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6714697406340058 2082
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup 0.8571428571428572 14
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6607851985559567 22160
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6644025933930225 3239
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup 0.6666666666666667 12
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.6448727214325864 62265
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6719980902363333 4189
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6569881430305293 42844
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6366995073891626 6496
known_articleID-known_customerID-known_voucherID-known_productGroup 0.6434649183986609 93197
